In [ ]:
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 복지시설, 대중교통 데이터 다운로드

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/02.모비젠/001.분석_저소득(2107_국토부)/03.분석진행/3. 소스 및 데이터 정리 /#3 구별 노인여가복지시설 현황/3. (Excel 변환파일) 복지시설_시군구_추가.csv", encoding='cp949')

In [ ]:
df.head()

,OBJECTID,id,fac_type,fac_nam,fac_area,sequence,addr,x,y,읍면동,시군구
0,3761,1,경로당,청 운,74.80,3761,종로구 필운대로 86 1층(신교동),126.967416,37.583506,서울특별시 종로구 신교동,서울특별시 종로구
1,3762,2,경로당,세종마을,98.30,3762,종로구 자하문로11길33(통인동),126.969958,37.580307,서울특별시 종로구 통인동,서울특별시 종로구
2,3763,3,경로당,사 직,94.31,3763,종로구사직로9길 15-25(사직동),126.965531,37.576482,서울특별시 종로구 사직동,서울특별시 종로구
3,3764,4,경로당,복 정,73.60,3764,종로구 삼청로4길 22(삼청동),126.982103,37.582389,서울특별시 종로구 삼청동,서울특별시 종로구
4,3765,5,경로당,삼 청,118.80,3765,종로구 삼청로5길 30(팔판동),126.980219,37.583037,서울특별시 종로구 팔판동,서울특별시 종로구


In [ ]:
df_bus = pd.read_csv("/content/drive/MyDrive/02.모비젠/001.분석_저소득(2107_국토부)/03.분석진행/3. 소스 및 데이터 정리 /#5 노인여가복지시설 대중교통 접근성 점수/0. (raw 데이터) 서울시버스정류소좌표데이터(2021.01.14.).csv", engine='python', encoding='cp949')

In [ ]:
df_bus.head()

,표준ID,ARS-ID,정류소명,X좌표,Y좌표
0,100000001,1001,종로2가사거리,126.987786,37.569764
1,100000002,1002,창경궁.서울대학교병원,126.996520,37.579179
2,100000003,1003,명륜3가.성대입구,126.998290,37.582709
3,100000004,1004,종로2가.삼일교,126.987507,37.568582
4,100000005,1005,혜화동로터리.여운형활동터,127.001694,37.586230


In [ ]:
df_train_1 = pd.read_excel("/content/drive/MyDrive/02.모비젠/001.분석_저소득(2107_국토부)/03.분석진행/3. 소스 및 데이터 정리 /#5 노인여가복지시설 대중교통 접근성 점수/1. (Python 변환파일) 서울교통공사_1호선~8호선_위경도추가.xlsx")

In [ ]:
df_train_1.head()

,Unnamed: 0,역번호,역사명,노선번호,노선명,영문 역사명,한자역사명,환승역 구분,환승 노선번호,환승 노선명,역위도,역경도,운영기관명,역사 도로명주소,역사전화번호,데이터기준일자,위도,경도
0,0,0124,청량리(서울시립대입구),I4101,1호선,Cheongnyangni (University of Seoul),NaN,도시철도 환승역,I4101,수도권 광역철도 1호선,NaN,NaN,서울특별시 서울교통공사,서울특별시 동대문구 왕산로 지하 205,NaN,2017-10-11,37.583796,127.053490
1,1,0125,제기동,I4101,1호선,Jegidong,NaN,도시철도 일반역,NaN,NaN,NaN,NaN,서울특별시 서울교통공사,서울특별시 동대문구 왕산로 지하 93,NaN,2017-10-11,37.578176,127.034591
2,2,0126,신설동,I4101,1호선,Sinseoldong,NaN,도시철도 환승역,I4101,수도권 광역철도 1호선,NaN,NaN,서울특별시 서울교통공사,서울특별시 동대문구 왕산로 지하 1,NaN,2017-10-11,37.575326,127.024850
3,3,0127,동묘앞,I4101,1호선,Dongmyo,NaN,도시철도 환승역,I4101,수도권 광역철도 1호선,NaN,NaN,서울특별시 서울교통공사,서울특별시 종로구 종로 359,NaN,2017-10-11,37.573246,127.016589
4,4,0128,동대문,I4101,1호선,Dongdaemun,NaN,도시철도 환승역,I4101,수도권 광역철도 1호선,NaN,NaN,서울특별시 서울교통공사,서울특별시 종로구 종로 지하 302,NaN,2017-10-11,37.571983,127.011708


In [ ]:
df_train_2 = pd.read_excel("/content/drive/MyDrive/02.모비젠/001.분석_저소득(2107_국토부)/03.분석진행/3. 소스 및 데이터 정리 /#5 노인여가복지시설 대중교통 접근성 점수/1. (Python 변환파일) 수도권9호선주소_위경도추가.xlsx")

In [ ]:
df_train_2.head()

,철도운영기관명,선명,역명,지번주소,도로명주소,위도,경도
0,서울9호선,9호선,개화,서울특별시 강서구 개화동 493-2,서울특별시 강서구 개화동로 8길 38,37.578464,126.796992
1,서울9호선,9호선,김포공항,서울특별시 강서구 방화동 886,서울특별시 강서구 하늘길 77,37.562402,126.801289
2,서울9호선,9호선,공항시장,서울특별시 강서구 공항동 산 20-2,서울특별시 강서구 방화동로 30,37.562315,126.808696
3,서울9호선,9호선,신방화,서울특별시 강서구 방화동 217-156,서울특별시 강서구 방화대로 301,37.567513,126.817353
4,서울9호선,9호선,마곡나루,서울특별시 강서구 마곡동 367-6,서울특별시 강서구 마곡중앙5로 2,37.566936,126.826561


# 복지시설별 대중교통 수 구하기

In [ ]:
pip install haversine

In [ ]:
from haversine import haversine

## 버스 정류장 수 구하기

In [ ]:
df['정류소명'] = ''
df['정류소 표준ID'] = ''
df['정류소 거리'] = ''
df['정류소 개수'] = ''

In [ ]:
# 각 복지시설에서 버스 정류소가 몇 개나 있는지를 확인 

for i in range(len(df)): # 복지시설 
    cnt = 0
    for j in range(len(df_bus)): # 대중교통시설 
        start = (df.loc[i,'y'], df.loc[i,'x']) # 복지시설 행 열 
        end = (df_bus.loc[j, 'Y좌표'], df_bus.loc[j,'X좌표']) # 대중교통시설 
        
        distance = haversine(start, end)
        if distance <= 0.5:
            cnt += 1
            df.loc[i, '정류소명'] += " " + df_bus.loc[j, '정류소명']
            df.loc[i,'정류소 표준ID'] += " " +  str(df_bus.loc[j,'표준ID'])
            df.loc[i,'정류소 거리'] +=  " " + str(distance)
            df.loc[i,'정류소 개수'] = cnt
    # print(i)

## 지하철 역 수 구하기

In [ ]:
df['역사명'] = ''
df['역번호'] = ''
df['역 거리'] = ''
df['역 개수'] = ''

In [ ]:
# (1호선 ~ 8호선) 복지시설별 지하철역 갯수 구하기 

for i in range(len(df)): # 복지시설 
    cnt = 0
    for j in range(len(df_train_1)): # 대중교통시설 
        start = (df.loc[i,'y'], df.loc[i,'x']) # 복지시설 행 열 
        end = (df_train_1.loc[j, '위도'], df_train_1.loc[j,'경도']) # 대중교통시설 
        
        distance = haversine(start, end)
        if distance <= 0.5:
            cnt += 1
            df.loc[i, '역사명'] += " " + df_train_1.loc[j, '역사명']
            df.loc[i,'역번호'] += " " + str(df_train_1.loc[j,'역번호'])
            df.loc[i,'역 거리'] += " " + str(distance)
            df.loc[i,'역 개수'] = cnt
    #print(i)

0
1
2
3
4
5
6
7


In [ ]:
# (9호선) 복지시설별 지하철역 갯수 구하기 

for i in range(len(df)): # 복지시설 
    cnt = 0
    for j in range(len(df_train_2)): # 대중교통시설 
        start = (df.loc[i,'y'], df.loc[i,'x']) # 복지시설 행 열 
        end = (df_train_2.loc[j, '위도'], df_train_2.loc[j,'경도']) # 대중교통시설 
        
        distance = haversine(start, end)
        if distance <= 0.5:
            cnt += 1
            df.loc[i, '역사명'] += " " + df_train_2.loc[j, '역사명']
            df.loc[i,'역번호'] += " " + str(df_train_2.loc[j,'역번호'])
            df.loc[i,'역 거리'] += " " + str(distance)
            df.loc[i,'역 개수'] = cnt
    #print(i)

In [ ]:
df.to_csv("2. (Python 변환파일) 복지시설_대중교통_카운팅.csv", index=False)